In [260]:
from faker import Faker
import pandas as pd
import random

fake = Faker(locale='en_US')      #instance of Faker for US addresses
fake_latam= Faker(locale='es_AR') #instance of Faker for Latin American addresses


Addresses_list = []  
Customers_list=[]
Payments_list=[]
Orders_list=[]

#createAddresses creates a list of customer addresses   (address_id=36 is the address of the first customer)
def createAddresses(Addresses_list,num_of_addresses):
   

    for i in range(36,36+num_of_addresses):  
        Address = {}  
        Address_ = {}
        Address_["address_id"] =i
        Address_["city"]=fake.random_element(elements=(
            "New York", "New York", "New York", 
            "Miami", "Miami", "Seattle", 
            "Lima", "Lima", "Buenos Aires")) 
        Address_["postal_code"] =fake.postcode()  
        if  Address_["city"]=="New York":
            Address_["country"]="USA"
        elif  Address_["city"]=="Miami":
              Address_["country"]="USA"
        elif  Address_["city"]=="Seattle":
              Address_["country"]="USA"
        elif  Address_["city"]=="Buenos Aires":
              Address_["country"]="Argentina"  
        elif  Address_["city"]=="Lima":
              Address_["country"]="Peru"  
            
        if Address_["country"]=="USA":          
           Address_["street_address"] =  Faker(locale='en_US').street_address()
        elif Address_["country"]=="Argentina":          
           Address_["street_address"] = fake_latam.street_address()
        elif Address_["country"]=="Peru":          
           Address_["street_address"] = fake_latam.street_address()  
            
        Address_["street_address_second_line"] = fake.building_number()
        Address_["last_update"] = str(fake.date_between_dates("-5y","+1w"))+" "+str(fake.time(pattern="%H:%M:%S"))

        
        #Placing all columns in correct order for our database
        Address["address_id"]=Address_["address_id"]
        Address["street_address"] =Address_["street_address"] 
        Address["street_address_second_line"] =Address_["street_address_second_line"]
        Address["city"]=Address_["city"]
        Address["postal_code"]= Address_["postal_code"]
        Address["country"]=Address_["country"]
        Address["last_update"] =Address_["last_update"]
        
        Addresses_list.append(Address) 
    return pd.DataFrame(Addresses_list)
    

def createCustomers(Customers_list,num_of_customers):
      # Initialize an empty dictionary
    for i in range(1, num_of_customers):
        Customer = {}
        Customer ["customer_id"]=i
        address_id = 35 + i
        Customer ["address_id"]=address_id  

        country = None       
        country =Addresses_list[i-1]["country"]  
      
        if country=="USA":        
           Customer ["first_name"] = fake.first_name()       
           Customer ["last_name"]  = fake.last_name_nonbinary()
            
        if country=="Argentina" or country=="Peru" :        
           Customer ["first_name"] = fake_latam.first_name()       
           Customer ["last_name"]  = fake_latam.last_name_nonbinary()  
                  
        Customer ["phone_number"]=fake.msisdn()
        Customer ["email"]=fake.ascii_free_email()
        Customer ["active"]=fake.random_element(elements=("true","true","true","true","false"))
        Customer ["last_update"]= str(fake.date_between_dates("-5y","+1w"))+" "+str(fake.time(pattern="%X"))
       
        Customers_list.append(Customer)  

    return pd.DataFrame(Customers_list)
    
def createPayments(Payments_list,num_of_payments):
    for i in range(1, num_of_payments):
      Payment={}
      Payment["payment_id"]=i  
      Payment["customer_id"]=Customers_list[i-1]["customer_id"]
        
      #according to cafe rules
      #payments can be created only by employees with the position cashier and waiter
      #emp_id's of all cashiers and waiters are:16,17,18,19,20,26,27,28,29,30   
        
      Payment["emp_id"]=  fake.random_element(elements=(16,16,16,17,17,18,19,20,26,27,28,29,29,30,30))
      Payment["payment_date"]=str(fake.date_between_dates("-5y","+1w"))+" "+str(fake.time(pattern="%X")) 
      Payment["amount"]= random.randrange(1, 30, 2)*random.randrange(1, 10, 3)*0.99
      Payment["payment_type"]=fake.random_element(elements=("visa","visa","visa","visa","master card","master card","amex","cash","cash","pay pal",))
      Payments_list.append(Payment)         
    return pd.DataFrame(Payments_list)


def createOrders(Orders_list,num_of_orders):
    item_id_test=0
    #adding 1 order with 1 item for each customer
    for i in range(1, num_of_orders):
        Order={}
        Order["order_id"]=i
        Order["item_id"]=fake.random_element(elements=(2,2,2,2,4,4,4,4,1,1,1,3,3,3,5,5,6,7,7)) 
        
      # test for function updateItemQuantity begins 
        if Order["order_id"]==1:
           item_id_test=Order["item_id"] 
       # test for function updateItemQuantity ends
       
       
       
        Order["customer_id"]=i
        Order["cafe_id"]=fake.random_element(elements=(1,2,3,4,5))
        Order["payment_id"]=i
        Order["delivery"]=fake.random_element(elements=("FALSE","FALSE","FALSE","FALSE","TRUE"))
        Order["delivery_address_id"]=Customers_list[i-1]["address_id"]       
        Order["created_at"]=str(fake.date_between_dates("-5y","+1w"))+" "+str(fake.time(pattern="%X"))
        Order["item_quantity"]=fake.random_element(elements=(1,1,1,1,1,1,1,1,2,2,2,2,2,3,3,4,5,6))
        Order["customer_modifications"] =fake.random_element(elements=(" "," "," "," ","Add extra utensils please",
                                                                       "No utensils"))
        if  Order["delivery"]=="TRUE":
         Order["delivery_notes"] =fake.random_element(elements=(" "," ","Leave at the reception",
                                                                "Contactless delivery.Leave at my doorsteps",
                                                                "Please dont call.Text instead"))                                                    
        #if it is not a delivery order then  we're filling up delivery note with blanks                                           
        if  Order["delivery"]=="FALSE": 
         Order["delivery_notes"] =" "                                                                          
        Orders_list.append(Order)
    
 
   #[] add additional item to 50% of the orders 
   # item_id_for_additional_line=item_id_test+2
    
    addNewItemToExistingOrder(Orders_list,1,8,77)
    #updateItemQuantity(Orders_list,1, 10, 79)
    addOneMoreItemLineToRandomExistingOrders(Orders_list)
    addOneMoreItemLineToRandomExistingOrders(Orders_list)
    return pd.DataFrame(Orders_list)
    
  
def addNewItemToExistingOrder(Orders_list,order_id,item_id,qty):
    print("inside addNewItemToExistingOrder")
    items_list=[] 
    order_id_exists="false"
    item_id_exists="false"
    
    new_order_line={}  
    
    #verifying that this order with order_id was already created
    for order in Orders_list:
        if order.get("order_id") == order_id:
            items_list.append(order["item_id"])
            order_id_exists="true"          
            new_order_line["order_id"]=order["order_id"]  
            new_order_line["customer_id"]=order["customer_id"]   
            new_order_line["cafe_id"]=order["cafe_id"]
            new_order_line["payment_id"]=order["payment_id"]
            new_order_line["delivery"]=order["delivery"]
            new_order_line["delivery_address_id"]=order["delivery_address_id"]
            new_order_line["created_at"]=order["created_at"]
            new_order_line["item_quantity"]=order["item_quantity"]  
            new_order_line["customer_modifications"]=order["customer_modifications"]
            new_order_line["delivery_notes"]=order["delivery_notes"]
                  
    if order_id_exists=="false":
        print("Order with order_id "+ str(order_id)+" was not created yet")
        return False
        
    #now lets verify that order with order_id doesn't have item with requested item_id yet
    elif order_id_exists=="true":
        for item_ in items_list:
            if item_==item_id:
                item_id_exists="true"
                print("Order with order_id "+ str(order_id)+" already contains item "+str(item_id))
                return False
     # if we successfully verified that order with order_id already exists 
     # and it doesn't contain item with requested item_id
     # then we append new line to the Orders_list 
    if  order_id_exists=="true" and item_id_exists=="false":   
            new_order_line["item_id"]=item_id
            new_order_line["item_quantity"]=qty       
            Orders_list.append(new_order_line) 
            return True

def addOneMoreItemLineToRandomExistingOrders(Orders_list):
                                                                        # list of all item_id in the same order
    numb_addtnl_items=int(len(Orders_list)/2)                                         #number of additions
    print("Will be adding: "+str(numb_addtnl_items)+" items")
    

    unique_order_ids=[]    #iterating through Orders_list and collecting all unique order_id's into unique_order_ids[]
    for order in Orders_list:
        if order.get("order_id") not in unique_order_ids:
            unique_order_ids.append(order.get("order_id"))  
    print("Unique order_id's: ")
    for item in unique_order_ids:
        print(item)
    number_of_unique_order_ids=len(unique_order_ids)
   
    for i in range(1, numb_addtnl_items):
        items_list=[] 
        new_order_line={}
        new_order_line["order_id"]=random.randint(1,number_of_unique_order_ids)                                      
        new_order_line["item_id"]=fake.random_element(elements=(2,2,2,2,4,4,4,4,1,1,1,3,3,3,5,5,6,7,7)) 
        new_order_line["item_quantity"]=fake.random_element(elements=(1,1,1,1,1,1,1,1,2,2,2,2,2,3,3,4,5,6))
        item_id_found="false"
                                                                                                              #Verifying if order with generated order_id already contains item with genrated item_id.
                                                                                                           #If yes,we'll be increasing qty.If no,we'll be adding new line order  
        
        for order in Orders_list:
             if order.get("order_id") == new_order_line["order_id"]:
                items_list.append(order.get("item_id"))
                #order_id_found="true" 
                 
        for item in items_list:
            if item==new_order_line["item_id"]:
                item_id_found="true"
                
        if item_id_found=="true":
            print("Order with order_id "+ str(new_order_line["order_id"])+" already contains item "+str(new_order_line["item_id"]))
            print("Updating quantity of item "+ str(new_order_line["item_id"])+" to "+str(new_order_line["item_quantity"]))
            print("Calling updateItemQuantity ")
            updateItemQuantity(Orders_list,new_order_line["order_id"], new_order_line["item_id"],new_order_line["item_quantity"])
            
        elif item_id_found=="false":
            print("Will add new item "+ str(new_order_line["item_id"])+" to order id "+ str(new_order_line["order_id"]))
            print("Calling addNewItemToExistingOrder ")
            addNewItemToExistingOrder(Orders_list,new_order_line["order_id"], new_order_line["item_id"],new_order_line["item_quantity"])
            
     
def updateItemQuantity(Orders_list,order_id, item_id, update_to_quantity):
    print("inside updateItemQuantity ")
    items_list=[] 
    order_id_exists="false"
    item_id_exists="false"
    
    #verifying that this order with order_id was already created
    for order in Orders_list:
        if order.get("order_id") == order_id:
            items_list.append(order["item_id"])
            order_id_exists="true"  
            
    if order_id_exists=="false":
        print("Order with order_id "+ str(order_id)+" was not created yet")
        return False

    
    #now lets verify that order with order_id already have item with requested item_id 
    elif order_id_exists=="true":
        for item_ in items_list:
            if item_==item_id:
                item_id_exists="true"               
     
    if item_id_exists=="false":
        print("Order with order_id "+ str(order_id)+" doesn't contains item "+str(item_id))
        return False
    
    for order in Orders_list:
        # Assuming 'order_id' and 'item_id' are keys in the dictionaries
        if order.get("order_id") == order_id and order.get("item_id") == item_id:
            order["item_quantity"] =  update_to_quantity
            return True  # Return True to indicate the update was successful     
    



    
data_addresses=createAddresses(Addresses_list,999)
data_addresses.to_csv('Addresses_of_customers.csv',index=False)

data_customers=createCustomers(Customers_list,999)
data_customers.to_csv('Customers.csv',index=False)

data_payments=createPayments(Payments_list,999)
data_payments.to_csv('Payments.csv',index=False)

data_orders=createOrders(Orders_list,999)
data_orders.to_csv('Orders.csv',index=False)
            


inside addNewItemToExistingOrder
Will be adding: 499 items
Unique order_id's: 
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
2